In [5]:
import json
import pandas as pd
from collections import defaultdict
from pathlib import Path

In [ ]:
input_file = "data/"

In [11]:
input_file = "data/WikiEvents/coref/train_coref.jsonl"
with open(input_file) as f:
    lines = f.read().splitlines()
    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']
    df_inter['json_element'].apply(json.loads)
    df = pd.json_normalize(df_inter['json_element'].apply(json.loads))
input_file = "data/WikiEvents/train.jsonl"
with open(input_file) as f:
    lines = f.read().splitlines()
    df_inter = pd.DataFrame(lines)
    df_inter.columns = ['json_element']
    df_inter['json_element'].apply(json.loads)
    df_we = pd.json_normalize(df_inter['json_element'].apply(json.loads))

FileNotFoundError: [Errno 2] No such file or directory: 'data/WikiEvents/coref/train_coref.jsonl'

## Coref Mapping

In [16]:
df_we = pd.read_json('data/WikiEvents/train.json').set_index('doc_id')
df = pd.read_json("data/WikiEvents/train_coref.json").set_index('doc_key')

coref_mapping = defaultdict(dict)
for doc_id, row in df.iterrows():
    for cluster in row.clusters:
        for item in cluster:
            coref_mapping[doc_id][item] = cluster
for doc_id, row in df_we.iterrows():
    for event in row.event_mentions:
        for arg in event['arguments']:
            if arg['entity_id'] not in coref_mapping[doc_id].keys():
                coref_mapping[doc_id][arg['entity_id']] = arg['entity_id']

with open("data/coref_mapping.json", "w") as f:
    json.dump(coref_mapping,f)

## Feasible Roles

In [2]:
event_types = []
for _,row in df_we.iterrows():
    for em in row.event_mentions:
        event_types.append(em['event_type'])
event_types = list(set(event_types))
kairos = pd.read_json("data/WikiEvents/Ontology/event_role_KAIROS.json")
kairos = kairos.transpose()
problems = []
for i in event_types:
    if i not in kairos.index:
        print(i)
        problems.append(i)
row1 = kairos.loc['Contact.RequestCommand.Unspecified']
row2 = kairos.loc['Contact.ThreatenCoerce.Unspecified']
row3 = kairos.loc['Contact.ThreatenCoerce.Unspecified']
row4 = kairos.loc['Contact.RequestCommand.Unspecified']
row1.name = 'Contact.RequestCommand.Broadcast'
row2.name = 'Contact.ThreatenCoerce.Correspondence'
row3.name = 'Contact.ThreatenCoerce.Broadcast'
row4.name = 'Contact.RequestCommand.Meet'
kairos = kairos.append([row1,row2,row3,row4])

feasible_roles = defaultdict(dict)
for idx,row in kairos.iterrows():
    feasible_roles[idx] = row.roles
with open("data/feasible_roles.json", "w") as f:
    json.dump(feasible_roles,f)

NameError: name 'df_we' is not defined